## Intro

This notebook uses the new [Gemini 2.0 Flash](https://cloud.google.com/vertex-ai/generative-ai/docs/gemini-v2#2.0-flash) to analyze arxiv research papers. We use the DeepSeek-R1 paper (https://arxiv.org/abs/2501.12948) as an example.

You can compare results to the manually-curated blog post about DeepSeek-R1 here: https://gonzoml.substack.com/p/deepseek-r1-open-model-with-reasoning.



## Preps


In [1]:
%pip install --upgrade --quiet google-genai

Here we set Google Cloud project information and create client.

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [5]:
GOOGLE_CLOUD_PROJECT = 'api-project-616886193868'    # !!! CHANGE IT FOR YOUR OWN PROJECT
GOOGLE_CLOUD_LOCATION = 'us-central1'                # !!! ... AND REGION

In [15]:
from google import genai
from google.genai.types import HttpOptions, Part
from IPython.display import display, Markdown

In [7]:
client = genai.Client(vertexai=True, project=GOOGLE_CLOUD_PROJECT, location=GOOGLE_CLOUD_LOCATION)

We use the Gemini 2.0 Flash model, namely version 001. More about Gemini 2.0 models [here](https://cloud.google.com/vertex-ai/generative-ai/docs/gemini-v2).

In [8]:
model_id = "gemini-2.0-flash-001"

## Loading the paper

Gemini 2.0 Flash supports [Document understanding](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/document-understanding) and can work with PDF files.

The PDF files need to be uploaded to a bucket in Google Cloud first. I did it manually outside of the notebook.

In [9]:
# DeepSeek-R1 paper, original source
paper_url_source = 'https://arxiv.org/pdf/2501.12948'

# the link we work with
paper_url = 'https://storage.googleapis.com/arxiv-papers-copy/2501.12948v1.pdf'

In [10]:
pdf_file = Part.from_uri(
    file_uri=paper_url,
    mime_type="application/pdf",
)

Later, when the [Context caching](https://cloud.google.com/vertex-ai/generative-ai/docs/context-cache/context-cache-overview) feature will work with Gemini 2.0 it would make sense to start using it. Right now, Context caching only supports Gemini 1.5.

## High-level summary

Let's test the Gemini 2.0 Flash capabilities and ensure everything works with PDF files.

Here we ask the model for a short summary of the paper:

In [11]:
prompt = """
You are a highly skilled document summarization specialist. Your task is to provide a concise executive summary of no more than 300 words. Please summarize the given document for a general audience.
"""

In [12]:
response = client.models.generate_content(
    model=model_id,
    contents=[pdf_file, prompt],
)

Notice how quickly the model processed the whole 22-pages file!

In [16]:
display(Markdown(response.text))

DeepSeek-AI introduces DeepSeek-R1, a novel approach to enhance reasoning capabilities in Large Language Models (LLMs) through reinforcement learning (RL). The research explores the potential of LLMs to develop reasoning skills without supervised fine-tuning (SFT), emphasizing self-evolution via RL.

The study starts with DeepSeek-R1-Zero, directly applying RL to a base model. It demonstrates remarkable reasoning behaviors emerging naturally through the RL process. This pure RL approach validates that LLMs can be incentivized to reason effectively without prior SFT, marking a significant milestone.

To address limitations of DeepSeek-R1-Zero, like readability and language mixing, DeepSeek-R1 incorporates a multi-stage training pipeline, including a small amount of cold-start data before RL. This involves collecting high-quality data for initial fine-tuning, followed by reasoning-oriented RL. Furthermore, new SFT data is created using rejection sampling, enhancing capabilities in various domains. After which the model undergoes another RL phase to consider prompts from all scenarios. DeepSeek-R1 achieves performance comparable to OpenAI-01-1217 on reasoning tasks.

The researchers further investigate distilling the reasoning patterns from DeepSeek-R1 into smaller, more efficient models. Distillation proves more effective than directly applying RL to smaller base models, highlighting the importance of reasoning patterns discovered by larger models. DeepSeek-AI has made DeepSeek-R1-Zero, DeepSeek-R1, and six dense models, distilled from DeepSeek-R1, openly accessible to the research community.


The summary is quite good!

It is definitely more useful than the paper's own abstract:

> "We introduce our first-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1. DeepSeek-R1-Zero, a model trained via large-scale reinforcement learning (RL) without supervised fine-tuning (SFT) as a preliminary step, demonstrates remarkable reasoning capabilities. Through RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and intriguing reasoning behaviors. However, it encounters challenges such as poor readability, and language mixing. To address these issues and further enhance reasoning performance, we introduce DeepSeek-R1, which incorporates multi-stage training and cold-start data before RL. DeepSeek-R1 achieves performance comparable to OpenAI-o1-1217 on reasoning tasks. To support the research community, we open-source DeepSeek-R1-Zero, DeepSeek-R1, and six dense models (1.5B, 7B, 8B, 14B, 32B, 70B) distilled from DeepSeek-R1 based on Qwen and Llama."

I especially like that the model highlighted an important point I also highlighted. It is that distillation proved to be more effective than direct RL.







## More specific questions to the paper

Let's ask more questions to the paper.

I've prepared a set of general questions worth considering when you analyze a new paper. Feel free to adjust the list as you need. These questions give a broad overview and assume some amount of reasoning.

In the future, when the [Gemini 2.0 Flash Thinking](https://cloud.google.com/vertex-ai/generative-ai/docs/thinking#thoughts) model will support document understanding, it would be interesting to compare the results.

In [17]:
prompt = """
You are a highly skilled PhD-level AI/ML expert. Your task is to provide a concise answers of no more than 300 words. Please answer the following question for the professional audience: {question}
"""

In [20]:
questions = [
  {
    "category": "motivation_and_context",
    "question": "What is the central problem or challenge the paper aims to address?"
  },
  {
    "category": "motivation_and_context",
    "question": "Why is this problem important and what gap in the current literature does the paper fill?"
  },
  {
    "category": "contributions_and_novelty",
    "question": "What are the key contributions and innovations introduced in the paper?"
  },
  {
    "category": "contributions_and_novelty",
    "question": "How does the proposed approach differ from or improve upon previous methods?"
  },
  {
    "category": "methodology_and_technical_approach",
    "question": "What is the overall framework or algorithm used in this work?"
  },
  {
    "category": "methodology_and_technical_approach",
    "question": "Are the technical details (e.g., training methods, architecture, optimization strategies) clearly explained and justified?"
  },
  {
    "category": "methodology_and_technical_approach",
    "question": "How does the paper balance theoretical insights with practical implementation (e.g., use of reinforcement learning, supervised fine-tuning, or a hybrid approach)?"
  },
  {
    "category": "experimental_setup_and_evaluation",
    "question": "What datasets, benchmarks, or tasks are used to validate the proposed method?"
  },
  {
    "category": "experimental_setup_and_evaluation",
    "question": "Are the experiments designed rigorously to support the authors' claims?"
  },
  {
    "category": "experimental_setup_and_evaluation",
    "question": "How does the performance compare with state-of-the-art methods or established baselines?"
  },
  {
    "category": "experimental_setup_and_evaluation",
    "question": "Are there any ablation studies or error analyses that help understand the factors driving the performance?"
  },
  {
    "category": "analysis_and_interpretability",
    "question": "Does the paper provide insights into why the proposed method works (e.g., through qualitative examples, 'aha moments', or emergent behaviors)?"
  },
  {
    "category": "analysis_and_interpretability",
    "question": "How well can the approach be interpreted or explained to a non-expert audience?"
  },
  {
    "category": "limitations_and_future_directions",
    "question": "What are the current limitations or potential weaknesses of the approach?"
  },
  {
    "category": "limitations_and_future_directions",
    "question": "Are there suggestions for future research or open questions that remain?"
  },
  {
    "category": "broader_impact_and_reproducibility",
    "question": "What are the broader implications of the work for the research community, industry, or society?"
  },
  {
    "category": "broader_impact_and_reproducibility",
    "question": "Are ethical, safety, or deployment considerations discussed in the paper?"
  },
  {
    "category": "broader_impact_and_reproducibility",
    "question": "Is the work reproducible? Are the code, data, and experimental settings sufficiently detailed or openly available?"
  }
]


In [21]:
from tqdm import tqdm

for i in tqdm(range(len(questions)), desc="Processing questions"):
    question_data = questions[i]
    if "answer" not in question_data:
        current_question = question_data["question"]
        response = client.models.generate_content(
            model=model_id,
            contents=[pdf_file, prompt.format(question=current_question)],
        )
        question_data["answer"] = response.text


Processing questions: 100%|██████████| 18/18 [02:02<00:00,  6.82s/it]


Again, notice the speed! 18 questions were processed in just two minutes. Just 7 seconds for a question across the 22-page document!

## Display digest

Now we are ready to display all the model answers and compare with the [human (my own) baseline](https://gonzoml.substack.com/p/deepseek-r1-open-model-with-reasoning).

In [22]:
def display_questions(questions):

    for question_data in questions:
        question = question_data["question"]
        answer = question_data.get("answer", "No answer available.")
        display(Markdown('##'+question))
        display(Markdown(answer))

In [23]:
display_questions(questions)

##What is the central problem or challenge the paper aims to address?

The DeepSeek-R1 paper addresses the challenge of **incentivizing robust reasoning capabilities in large language models (LLMs) without relying on extensive supervised fine-tuning (SFT)**. While SFT is a common technique, the authors explore whether powerful reasoning can emerge directly through reinforcement learning (RL).

The central problem is two-fold:

1.  **Can an LLM develop complex reasoning abilities through RL alone, bypassing the need for labeled reasoning data?** This is investigated with DeepSeek-R1-Zero, a model trained solely with RL.
2.  **How can an RL-trained LLM be further improved to achieve human-aligned reasoning and strong general capabilities?** This is addressed with DeepSeek-R1, which incorporates a small amount of "cold-start" data and a multi-stage RL pipeline.

The paper also delves into the problem of efficiently transferring reasoning capabilities to smaller models via distillation, allowing them to achieve better performance than RL-trained small models. Ultimately, the paper seeks to demonstrate a comprehensive methodology for developing highly capable reasoning LLMs through RL and distillation, reducing the reliance on resource-intensive supervised methods.


##Why is this problem important and what gap in the current literature does the paper fill?

This paper addresses a critical challenge in the field of large language models (LLMs): incentivizing reasoning capability without relying heavily on supervised fine-tuning (SFT). The problem is important because SFT, while effective, requires extensive human-annotated data, which is expensive and limits scalability and potentially introduces bias. Furthermore, relying primarily on SFT may stifle the model's ability to explore novel reasoning strategies independently.

The paper tackles this by introducing DeepSeek-R1, trained through reinforcement learning (RL), demonstrating remarkable reasoning capabilities without initial SFT. This underscores that reasoning can emerge organically through RL, unlocking significant potential for self-evolution in LLMs. Addressing the limitation of readability and language mixing of DeepSeek-R1-Zero, DeepSeek-R1 incorporates multi-stage training and cold-start data before RL.

The paper fills a significant gap in the current literature by:

1.  **Validating RL-Only Reasoning:** Explicitly demonstrating that pure RL can successfully incentivize sophisticated reasoning behaviors in LLMs, challenging the prevalent reliance on SFT.
2.  **A Novel Training Pipeline:** Introducing a multi-stage training pipeline, which allows the models to improve reasoning patters and align with human preferences.
3.  **Distillation Insights:** Demonstrating that distilling reasoning patterns from large, RL-trained models can lead to smaller, highly capable models, which are beneficial for downstream tasks and accessibility to a broader research community. The release of the distilled models allows researchers to continue to improve them.

In essence, the paper pushes the boundaries of LLM training by showcasing the power of RL in fostering autonomous reasoning and introducing new possibilities for creating more efficient reasoning models.


##What are the key contributions and innovations introduced in the paper?

...

This DeepSeek-R1 research, centered on incentivizing reasoning in LLMs through reinforcement learning, has significant implications:

**Research Community:** The findings demonstrate that strong reasoning capabilities can emerge organically in LLMs via RL without relying on initial supervised fine-tuning. This challenges the conventional wisdom and opens up a new avenue for exploration. The open-sourcing of the models and datasets enables further investigation into RL-driven reasoning, cold-start strategies, and distillation techniques.

**Industry:** The success of distillation in transferring reasoning abilities to smaller, more efficient models is highly relevant to industry. Deploying smaller models with strong reasoning capabilities is cost-effective and energy-efficient. The methodology offers a pathway to create specialized, high-performing models for specific reasoning-intensive applications (coding, mathematics, scientific reasoning). Furthermore, the robust multi-stage training pipeline and the discovery of key training components (cold-start data, language consistency reward) offers a valuable roadmap for companies aiming to build LLMs.

**Society:**  Improved reasoning capabilities in LLMs can lead to more reliable and accurate AI-driven tools. Education, problem-solving, and decision-making tasks can potentially be significantly improved by RL and distillation.  Furthermore, the focus on alignment with human preferences during reinforcement learning and improving the helpfulness and harmlessness aspects of these LLMs has the potential to improve AI interactions in daily life. 


##Are ethical, safety, or deployment considerations discussed in the paper?

Yes, ethical, safety, and deployment considerations are discussed in the DeepSeek-R1 paper.

**Ethical Considerations:** The paper acknowledges the potential for language mixing issues, particularly when handling queries in languages other than Chinese and English. There is a discussion that DeepSeek-R1 might default to English for reasoning and responses, even if the original query was in another language. The authors also acknowledge potential safety RL related refusals to answer specific queries, demonstrating the importance of responsible AI development.

**Safety Considerations:** The authors emphasize aligning the model with human preferences through a secondary reinforcement learning stage, explicitly aiming to improve the model's helpfulness and harmlessness. They evaluate the model's entire response, including reasoning and summaries, to identify and mitigate potential risks, biases, or harmful content.

**Deployment Considerations:**
The paper highlights DeepSeek-R1's sensitivity to prompting styles, noting that few-shot prompting can degrade performance. They recommend that users directly describe the problem and specify the output format using a zero-shot setting for optimal results. This guidance aims to ensure users can effectively leverage the model's capabilities. Additionally, the discussion of distilling the reasoning capability to small dense models and open-sourcing them shows the commitment to the community.


##Is the work reproducible? Are the code, data, and experimental settings sufficiently detailed or openly available?

Based on the paper "DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via Reinforcement Learning," here's an assessment of reproducibility:

**Strengths (Towards Reproducibility):**

*   **Model Release:** The authors open-source DeepSeek-R1-Zero, DeepSeek-R1, and distilled models (1.5B-70B parameters). This is a major boon for reproducibility.
*   **Algorithm Detail:** GRPO is described with supporting mathematical equations, the reinforcement learning algorithm is adequately detailed.
*   **Training Data Details:** A description for creating the cold start, SFT, and RL data. The data is fine-tuned starting with the DeepSeek-V3-Base model.
*   **Benchmark Datasets:** Standard benchmarks used for evaluation are publicly accessible (MMLU, GPQA, etc.). Evaluation prompts can be derived from DeepSeek-V3.

**Weaknesses (Hindering Reproducibility):**

*   **Complete Experimental Settings:** Though some settings are specified, a full comprehensive disclosure of exact hyperparameters and training configurations would be highly valuable.
*   **Reward model details:** The rule-based reward models and the generative reward model using DeepSeek-V3 are lacking.
*   **Computational Resources:** The scale of training (model sizes, hardware) is not fully specified, making replication challenging for researchers with limited resources.

**Conclusion:**

The release of models is a major strength. However, full reproducibility would benefit from a complete specification of the training configuration and specifics of the training data and reward models. Researchers can use the released checkpoints for downstream applications but may struggle to precisely replicate the training process from scratch.

## Next steps

Along the notebook I already mentioned some steps worth doing. Let's gather everything here and add a few more points.

* [Context caching](https://cloud.google.com/vertex-ai/generative-ai/docs/context-cache/context-cache-overview) may help save money and even more time. Right now it only supports Gemini 1.5 family, so waiting for the updates.
* [Gemini 2.0 Flash Thinking](https://cloud.google.com/vertex-ai/generative-ai/docs/thinking#thoughts) is a reasoning model similar to OpenAI's o1/o3, Qwen QwQ, or DeepSeek-R1. At the moment, it is experimental and does not support document understanding and PDF inputs. As some of our questions assume reasoning, we may expect even better results with this model.
* Processing multiple papers at once. As DeepSeek-R1 paper is built on top of other DeepSeek papers (like DeepSeek-V2 and V3, DeepSeekMath, DeepSeekMoE), it's worth exploring multi-document questions, as more interesting details are in other papers.
* More automation along the notebook, e.g. extracting relevant links to other papers, automatically upload papers to a bucket, etc.
* Multi-agent pipeline with more specialization along the way. One agent may extract relevant parts of the text, another one may check the code, others may validate the findings, prepare blog posts and publish them.


Google Cloud credits are provided for this project.

Author: [Grigory Sapunov](https://www.linkedin.com/in/grigorysapunov/), AI/ML GDE

February 23, 2025